This notebook contains examples of various basic tasks using the [Socrata Publishing API](https://dev.socrata.com/publishers/getting-started.html). The principles we'll discuss apply across many languages, and there are [libraries available for most of the popular ones](https://dev.socrata.com/libraries/). For these examples, we'll be using xmunoz's community-built [sodapy](https://github.com/xmunoz/sodapy) library for Python 3.

## Authentication

Socrata's API relies on a standard username/password combination, with an optional (but highly-recommended) app token. How you choose to store these credentials is up to you; you simply need to provide them, usually just once per session. For this example, we're using [environment variables](https://en.wikipedia.org/wiki/Environment_variable), meaning that the operating system is storing them in memory and we just need to ask for them.

Creating environment variables varies a lot by platform; it's different for [Windows](https://kb.wisc.edu/cae/page.php?id=24500), [Mac OS X](http://osxdaily.com/2015/07/28/set-enviornment-variables-mac-os-x/) and [Linux](https://www.digitalocean.com/community/tutorials/how-to-read-and-set-environmental-and-shell-variables-on-a-linux-vps). For now, let's assume that we have three such environment variables available to us:

`SODA_USERNAME`; `SODA_PASSWORD`; `SODA_APP_TOKEN`

Here's how we access them in Python:

In [1]:
import os

os.getenv('SODA_USERNAME', 'username goes here')

'abraham.epton@socrata.com'

We won't display the others here, but let's assume they also exist. Using `sodapy` (which has [great documentation](https://github.com/xmunoz/sodapy)), let's create a client that lets us perform operations on a specific instance of the Socrata platform. In this case, it's a testing domain we use for education and training.

In [2]:
from sodapy import Socrata

# Extract our credentials from the environment
username = os.getenv('SODA_USERNAME', 'username goes here')
password = os.getenv('SODA_PASSWORD', 'password goes here')
app_token = os.getenv('SODA_APP_TOKEN', 'app token goes here')

# Specify which Socrata domain we'd like to talk to
domain = 'edu.demo.socrata.com'

client = Socrata(domain, app_token, username=username, password=password)

# What do we have here?
print(client)

Great! Now we've got a client object that we can use to upload, download and modify datasets on the platform at [edu.demo.socrata.com](edu.demo.socrata.com).

## Creating a dataset

In order to create a dataset, you really only need to give Socrata the name you'd like to use. Often, we find it helpful to create some columns as well. To do so, pass in a list of the columns, along with their name and type, as below.

We're also telling the platform which column to use as a *row identifier* - this lets the platform know which row we're talking about. Row identifiers should be unique in a particular dataset, though beyond that constraint they can be basically anything you like. Names aren't good row identifiers, usually, because they're rarely truly unique, but in this example our woodchucks can be guaranteed to have unique names, so we'll use that. (You don't need to specify a row identifier, unless you plan on using the *upsert* method we discuss in a little bit.)

Note that when we specify which column is the row identifier, we use a slightly different version of the column name - `woodchuck_name` instead of `"Woodchuck name"`. This is because the API typically translates the human-readable display names into something easier for a machine to handle, so it makes everything lowercase and translates spaces to \_s (among other things).

This is just scratching the surface of what you can do when creating a dataset; the full set of options is described in more detail [in our documentation](https://dev.socrata.com/publishers/getting-started.html).

In [3]:
# Create the columns you'd like this dataset to have
columns = [
  {
    'name': 'Woodchuck name',
    'dataTypeName': 'text'
  },
  {
    'name': 'Amount of wood chucked',
    'dataTypeName': 'number'
  }
]

# Now actually create the dataset
result = client.create('Wood chucked by various woodchucks', columns=columns, row_identifier='woodchuck_name')

When the dataset is created, the response from the API will include the id (sometimes referred to as a four-by-four, since it's always 4 alphanumeric characters, then a hyphen, then another 4 alphanumeric characters). You'll need this id for all future operations using this dataset. Let's see what we get back from the API when we successfully create a dataset:

In [4]:
print(result)

{'id': '6rpc-z7vc', 'name': 'Wood chucked by various woodchucks', 'averageRating': 0, 'createdAt': 1515020840, 'downloadCount': 0, 'hideFromCatalog': False, 'hideFromDataJson': False, 'newBackend': False, 'numberOfComments': 0, 'oid': 27646203, 'provenance': 'official', 'publicationAppendEnabled': False, 'publicationGroup': 14761964, 'publicationStage': 'unpublished', 'rowIdentifierColumnId': 338297198, 'rowsUpdatedAt': 1515020840, 'rowsUpdatedBy': '5sfv-yvt6', 'tableId': 14761964, 'totalTimesRated': 0, 'viewCount': 0, 'viewLastModified': 1515020840, 'viewType': 'tabular', 'columns': [{'id': 338297198, 'name': 'Woodchuck name', 'dataTypeName': 'text', 'fieldName': 'woodchuck_name', 'position': 1, 'renderTypeName': 'text', 'tableColumnId': 59270136, 'format': {}}, {'id': 338297199, 'name': 'Amount of wood chucked', 'dataTypeName': 'number', 'fieldName': 'amount_of_wood_chucked', 'position': 2, 'renderTypeName': 'number', 'tableColumnId': 59270137, 'format': {}}], 'metadata': {'rowIdenti

What we really care about is the dataset id, so let's save that for future use:

In [5]:
dataset_id = result.get('id', 'no dataset id found')

When we first create a dataset, it's in *working copy* mode, meaning it's not yet public. To make it public, just publish it like so:

In [6]:
publication_result = client.publish(dataset_id)

## Updating a dataset

Now that we've got a dataset, let's add some data to it. There are two strategies we can use for this: *replace* and *upsert*.

When we replace a dataset, we do just what the name suggests: delete all the data that's currently there, and replace it with whatever we send along.

When we upsert something on a dataset, we only change specific rows and leave the rest alone (this is quite useful for large datasets; imagine having to change a typo in one row of a million-row dataset). If the row we're upserting doesn't exist yet, we add it. (Hence the name: update + insert = upsert)

Let's start by adding some data to the currently-empty dataset of wood chucked by woodchucks:

In [7]:
# Generate the replacement data - a list of objects, each one describing a row to be added
data = [
  {
    'Woodchuck name': 'Tommy',
    'Amount of wood chucked': 30
  },
  {
    'Woodchuck name': 'Natalie',
    'Amount of wood chucked': 12
  }
]

# Just pass the list of replacement rows and the dataset id to the API, and let it do the rest
client.replace(dataset_id, data)

{'By RowIdentifier': 2,
 'By SID': 0,
 'Errors': 0,
 'Rows Created': 2,
 'Rows Deleted': 0,
 'Rows Updated': 0}

Hey, look at that - we got some statistics back! The platform tells us how many rows we changed, and by which method.

Now let's take a look at upserting. It works almost exactly the same as replacing a dataset does, but under the hood, the platform looks for row identifiers and only changes rows that already exist, inserting the rest.

In [8]:
# Generate the upsertable data - a list of objects, each one describing a row to be added. Make sure to pay attention
# to the row identifier!
data = [
  {
    'Woodchuck name': 'Natalie',
    'Amount of wood chucked': 250
  },
  {
    'Woodchuck name': 'Hercules',
    'Amount of wood chucked': 3
  }
]

# Just pass the list of replacement rows and the dataset id to the API, and let it do the rest
# Note that we're explicitly calling upsert() and not replace()
client.upsert(dataset_id, data)

{'By RowIdentifier': 2,
 'By SID': 0,
 'Errors': 0,
 'Rows Created': 1,
 'Rows Deleted': 0,
 'Rows Updated': 1}

As you can see from the statistics, we modified 2 rows by row identifier: 1 row was updated and 1 was created, since the row identifier wasn't already present in the dataset.

## Deleting a row or dataset

Ashes to ashes, dust to dust: no example walkthrough would be complete without acknowleding the cycle of data, from birth to death. We have created datasets and rows, and now we must destroy them.

First, let's delete a specific row. Again, we use the row identifier, and poor Hercules the Woodchuck's wood-chucking efforts will no longer be available for the world to examine:

In [9]:
client.delete(dataset_id, row_id='Hercules')

<Response [200]>

And then, of course, we can delete the entire dataset, should we desire:

In [10]:
client.delete(dataset_id)

<Response [200]>

Since we got an HTTP code 200, our request was successful, and the dataset has been deleted. Thanks for reading!